# K近傍法により欠損値を補完する

In [127]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer

In [171]:
filepath = '../../data/raw/df.xlsx'
df = pd.read_excel(filepath).dropna()
df = df[df['year'] >= 2000]
df

,island,year,island_id,region_code,region_name,prefecture_code,prefecture_code.1,population,dummy_island_has_bridge,dummy_island_is_connected_mainland,year_bridge_opened,dummy_after_bridge_opened,year_connect_mainland,dummy_after_connect_mainland,income
5,島後,2005.0,0.0,32528.0,隠岐の島町,32.0,島根県,17259.0,0.0,0.0,0.0,0.0,0.0,0.0,19083931.0
6,島後,2006.0,0.0,32528.0,隠岐の島町,32.0,島根県,17016.0,0.0,0.0,0.0,0.0,0.0,0.0,18912797.0
7,島後,2007.0,0.0,32528.0,隠岐の島町,32.0,島根県,16779.0,0.0,0.0,0.0,0.0,0.0,0.0,17833873.0
8,島後,2008.0,0.0,32528.0,隠岐の島町,32.0,島根県,16417.0,0.0,0.0,0.0,0.0,0.0,0.0,17127603.0
9,島後,2009.0,0.0,32528.0,隠岐の島町,32.0,島根県,16099.0,0.0,0.0,0.0,0.0,0.0,0.0,16606138.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3076,鵜来島,2016.0,162.0,39208.0,宿毛市,39.0,高知県,41.0,0.0,0.0,0.0,0.0,0.0,0.0,19393764.0
3077,鵜来島,2017.0,162.0,39208.0,宿毛市,39.0,高知県,41.0,0.0,0.0,0.0,0.0,0.0,0.0,19737157.0
3078,鵜来島,2018.0,162.0,39208.0,宿毛市,39.0,高知県,38.0,0.0,0.0,0.0,0.0,0.0,0.0,20334583.0
3079,鵜来島,2019.0,162.0,39208.0,宿毛市,39.0,高知県,38.0,0.0,0.0,0.0,0.0,0.0,0.0,20970351.0


In [178]:
pivoted_df = df.pivot(index='year', columns='island_id', values=['population', 'dummy_after_bridge_opened', 'income'])
pivoted_df

population                  ...      income                        
island_id      0.0     1.0     2.0    ...       161.0       162.0       163.0
year                                  ...                                    
2000.0           NaN  2672.0  3804.0  ...         NaN         NaN         NaN
2001.0           NaN  2675.0  3875.0  ...         NaN  23245303.0  23245303.0
2002.0           NaN  2631.0  3817.0  ...         NaN  22413480.0  22413480.0
2003.0           NaN  2581.0  3738.0  ...         NaN  22594682.0  22594682.0
2004.0           NaN  2514.0  3679.0  ...         NaN  21214656.0  21214656.0
2005.0       17259.0  2406.0  3607.0  ...         NaN  21048350.0  21048350.0
2006.0       17016.0  2492.0  3567.0  ...         NaN  21029470.0  21029470.0
2007.0       16779.0  2425.0  3434.0  ...         NaN  20207587.0  20207587.0
2008.0       16417.0  2403.0  3390.0  ...         NaN  19351610.0  19351610.0
2009.0       16099.0  2370.0  3334.0  ...         NaN  18831105.0  18831105.0
2010.0       15930.0  2378.0  3238.0  ...  18144578.0  18144578.0  18144578.0
2011.0       15738.0  2321.0  3231.0  ...         NaN  18415966.0  18415966.0
2012.0       15440.0  2288.0  3168.0  ...         NaN  18584283.0  18584283.0
2013.0       15296.0  2297.0  3140.0  ...         NaN  18575052.0  18575052.0
2014.0       15038.0  2339.0  3080.0  ...         NaN  18920959.0  18920959.0
2015.0       14901.0  2304.0  3017.0  ...         NaN  19088873.0  19088873.0
2016.0       14654.0  2298.0  2923.0  ...         NaN  19393764.0  19393764.0
2017.0       14550.0  2225.0  2879.0  ...         NaN  19737157.0  19737157.0
2018.0       14337.0  2236.0  2868.0  ...         NaN  20334583.0  20334583.0
2019.0       14116.0  2222.0  2811.0  ...         NaN  20970351.0  20970351.0
2020.0       13882.0  2184.0  2758.0  ...  20781354.0  20781354.0  20781354.0

[21 rows x 459 columns]

In [182]:
pivoted_df['dummy_after_bridge_opened']

island_id,0.0,1.0,2.0,3.0,4.0,6.0,7.0,8.0,9.0,10.0,11.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,28.0,29.0,31.0,32.0,33.0,35.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,46.0,...,123.0,124.0,125.0,126.0,127.0,128.0,129.0,130.0,131.0,132.0,133.0,134.0,135.0,136.0,137.0,138.0,139.0,140.0,141.0,142.0,143.0,144.0,145.0,146.0,147.0,148.0,149.0,150.0,151.0,152.0,153.0,154.0,155.0,156.0,157.0,158.0,160.0,161.0,162.0,163.0
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,NaN,...,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0
2002.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,NaN,...,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0
2003.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,NaN,...,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0
2004.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,NaN,...,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0
2005.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,NaN,...,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0
2006.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,NaN,...,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0
2007.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,NaN,...,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0
2008.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,1.0,0.0,NaN,...,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0


In [176]:
# Step 4: Apply KNNImputer
imputer = KNNImputer(n_neighbors=3)
imputed_data = imputer.fit_transform(pivoted_df)

imputed_data

# Step 5: Convert back to dataframe
imputed_df = pd.DataFrame(imputed_data, columns=pivoted_df.columns, index=pivoted_df.index)

imputed_df['population']

island_id,0.0,1.0,2.0,3.0,4.0,6.0,7.0,8.0,9.0,10.0,11.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,28.0,29.0,31.0,32.0,33.0,35.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,46.0,...,123.0,124.0,125.0,126.0,127.0,128.0,129.0,130.0,131.0,132.0,133.0,134.0,135.0,136.0,137.0,138.0,139.0,140.0,141.0,142.0,143.0,144.0,145.0,146.0,147.0,148.0,149.0,150.0,151.0,152.0,153.0,154.0,155.0,156.0,157.0,158.0,160.0,161.0,162.0,163.0
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000.0,14334.333333,2672.0,3804.0,718.0,4583.000000,16.333333,134.0,494.333333,46.666667,425.0,217.000000,66.333333,118.333333,5.333333,11.666667,124.666667,690.333333,1362.333333,332.0,33.333333,154.0,89.333333,764.0,3706.000000,679.666667,16710.0,58.333333,28187.000000,1028.666667,9606.000000,20.666667,32.5,37.0,9095.333333,20.0,3233.333333,42.333333,1798.000000,28.333333,139.0,...,2981.666667,614.666667,2.000000,1973.666667,2228.666667,2.000000,5319.0,6037.0,41.333333,5693.0,38.0,655.666667,56.0,129.333333,31.666667,30.666667,28.333333,4.333333,318.666667,29.666667,1132.666667,87.666667,187.666667,366.0,3853.333333,587.666667,499.0,213.333333,36.0,360.666667,1316.333333,158.0,427.0,467.0,60.333333,62.0,322.0,502.0,55.0,271.0
2001.0,15654.666667,2675.0,3875.0,748.0,4165.666667,17.000000,160.0,506.000000,42.000000,425.0,193.333333,80.000000,143.000000,6.000000,15.000000,151.000000,820.000000,1692.000000,412.0,41.000000,210.0,101.000000,909.0,3401.000000,811.000000,15795.0,70.000000,26764.666667,1211.000000,9138.333333,20.666667,32.5,48.0,10090.000000,20.0,3445.000000,52.000000,1836.000000,34.000000,139.0,...,3154.000000,695.000000,2.000000,2215.000000,2336.000000,1.333333,5319.0,6037.0,51.000000,5693.0,50.0,667.000000,73.0,162.000000,39.000000,44.000000,35.000000,6.000000,393.000000,40.000000,1232.333333,105.000000,231.000000,450.0,4529.000000,734.000000,613.0,265.000000,53.0,431.000000,1348.000000,204.0,507.0,551.0,79.000000,62.0,322.0,502.0,64.0,328.0
2002.0,15491.333333,2631.0,3817.0,757.0,4165.666667,17.000000,159.0,493.000000,41.000000,425.0,193.333333,74.000000,137.000000,6.000000,14.000000,143.000000,786.000000,1615.000000,398.0,39.000000,184.0,105.000000,897.0,3401.000000,769.000000,15795.0,70.000000,26764.666667,1189.000000,9138.333333,20.666667,32.5,47.0,9847.000000,23.0,3367.000000,52.000000,1793.000000,36.000000,139.0,...,3168.000000,671.000000,2.000000,2185.000000,2312.000000,1.333333,5319.0,6037.0,47.000000,5693.0,49.0,657.000000,72.0,157.000000,35.000000,41.000000,35.000000,5.000000,384.000000,36.000000,1232.333333,105.000000,227.000000,437.0,4428.000000,694.000000,599.0,264.000000,46.0,428.000000,1316.000000,198.0,481.0,542.0,76.000000,62.0,322.0,502.0,63.0,332.0
2003.0,15654.666667,2581.0,3738.0,784.0,4165.666667,15.000000,153.0,484.000000,39.000000,425.0,193.333333,72.000000,126.000000,6.000000,13.000000,139.000000,770.000000,1546.000000,390.0,39.000000,186.0,93.000000,880.0,3401.000000,744.000000,15795.0,68.000000,26764.666667,1153.000000,9138.333333,20.666667,32.5,45.0,9681.000000,23.0,2888.000000,48.000000,1765.000000,39.000000,139.0,...,3148.000000,668.000000,2.000000,2118.000000,2290.000000,1.333333,5319.0,6037.0,43.000000,5693.0,45.0,643.000000,62.0,153.000000,34.000000,40.000000,34.000000,5.000000,373.000000,38.000000,1232.333333,103.000000,225.000000,417.0,4340.000000,677.000000,570.0,252.000000,44.0,415.000000,1285.000000,192.0,481.0,514.0,75.000000,62.0,322.0,502.0,62.0,317.0
2004.0,16309.000000,2514.0,3679.0,766.0,4165.666667,13.000000,150.0,468.000000,42.000000,425.0,193.333333,74.000000,126.000000,6.000000,12.000000,143.000000,771.000000,1468.000000,362.0,37.000000,177.0,90.000000,866.0,3401.000000,726.000000,15795.0,69.000000,26764.666667,1119.000000,9138.333333,20.666667,32.5,44.0,9513.000000,21.0,2829.000000,49.000000,1708.000000,34.000000,139.0,...,3060.000000,658.000000,2.000000,2055.000000,2261.000000,2.000000,5319.0,6037.0,41.000000,5693.0,43.0,601.000000,54.0,142.000